# DINO Animal Detection - Zapovednik

**Инструкция:**
1. Запустите ячейки 1-3 для установки
2. Загрузите свой датасет (ячейка 4)
3. Запустите анализ (ячейка 5)

#@title 1. Клонирование и установка
!git clone https://github.com/YanGaev2/dino-detection.git
%cd dino-detection

!pip install -q torch torchvision --index-url https://download.pytorch.org/whl/cu121
!pip install -q easyocr pycocotools scikit-learn timm

print("[OK] Установка завершена")

In [ ]:
#@title 2. Скачивание чекпоинта
import os
os.makedirs("output_dino", exist_ok=True)

# Скачиваем с Яндекс.Диска
!curl -L "https://getfile.dokpub.com/yandex/get/https://disk.yandex.ru/d/Qer6MIgthiA7Ww" -o output_dino/checkpoint.pth

size_mb = os.path.getsize("output_dino/checkpoint.pth") / (1024*1024)
print(f"[OK] Чекпоинт: {size_mb:.1f} MB")

In [ ]:
#@title 3. Сборка CUDA
%cd DINO/models/dino/ops
!python setup.py build install
%cd ../../../..
print("[OK] CUDA собран")

In [ ]:
#@title 4. Загрузка датасета (zip архив)
from google.colab import files
import zipfile

print("Загрузите zip архив с датасетом:")
uploaded = files.upload()

for fn in uploaded.keys():
    with zipfile.ZipFile(fn, 'r') as z:
        z.extractall('dataset')
    print(f"[OK] Распаковано в dataset/")

!ls dataset/

In [ ]:
#@title 5. Запуск анализа
#@markdown Формат датасета:
dataset_format = "COCO" #@param ["COCO", "YOLO", "images"]
#@markdown Путь к датасету:
dataset_path = "dataset" #@param {type:"string"}
#@markdown Порог уверенности:
threshold = 0.3 #@param {type:"slider", min:0.1, max:0.9, step:0.05}

!python run_analysis.py --checkpoint output_dino/checkpoint.pth --dataset_path {dataset_path} --format {dataset_format} --threshold {threshold}

#@title 6. Просмотр результатов
import pandas as pd

print("="*60 + "\nСВОДНАЯ ТАБЛИЦА ПО ВИДАМ\n" + "="*60)
display(pd.read_csv("results/summary_by_species.csv"))

print("\n" + "="*60 + "\nДЕТАЛЬНЫЙ ЛОГ (первые 20)\n" + "="*60)
display(pd.read_csv("results/detailed_detections.csv").head(20))

In [ ]:
#@title 7. Скачать результаты
from google.colab import files
!zip -r results.zip results/
files.download('results.zip')

---
**Готово!** Результаты в папке `results/`:
- `summary_by_species.csv` - сводка по видам
- `detailed_detections.csv` - детальный лог
- `analysis_results.json` - полные данные + ROC AUC